In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from MEDIC.secrets import controller

In [3]:
def get_delta_df(df, target: str):
    df = df.set_index(['date', 'CountryCode'])\
                .unstack(level=-1)\
                .diff(axis=0)\
                .unstack(level=0)\
                .reset_index()\
                .rename(columns={0: target})\
                .drop('level_0', axis=1)\
                .dropna()
    df['date'] = pd.to_datetime(df['date'])

    return df

In [4]:
confirmed_df = controller.get_df_from_sql("select CountryCode, date, confirmed "
                                          "from covid_info.COVID_Cases "
                                          "where SubdivisionCode is null and CountryCode != 'not found'")
confirmed_df = get_delta_df(confirmed_df, 'confirmed')

deaths_df = controller.get_df_from_sql("select CountryCode, date, deaths "
                                          "from covid_info.COVID_Cases "
                                          "where SubdivisionCode is null and CountryCode != 'not found'")
deaths_df = get_delta_df(deaths_df, 'deaths')

recovered_df = controller.get_df_from_sql("select CountryCode, date, recovered "
                                          "from covid_info.COVID_Cases "
                                          "where SubdivisionCode is null and CountryCode != 'not found'")
recovered_df = get_delta_df(recovered_df, 'recovered')

covid_df = confirmed_df.merge(deaths_df, on=['CountryCode', 'date'])
covid_df = covid_df.merge(recovered_df, on=['CountryCode', 'date'])
covid_df = covid_df.rename(columns={
    'confirmed': 'covid_confirmed',
    'deaths': 'covid_deaths',
    'recovered': 'covid_recovered',
})

In [5]:
testing_df = controller.get_df_from_sql(
    "select `ISO code`, Date, `Cumulative total`, `7-day smoothed daily change`, `Short-term positive rate`, `Short-term tests per case` "
    "from covid_info.OWID_testing_data"
).rename(columns={
    'ISO code': 'CountryCode',
    'Date': 'date',
    '7-day smoothed daily change': 'test_7dayMA', 
    'Short-term positive rate': 'test_shortPosRate', 
    'Short-term tests per case': 'test_shortTestperCase', 
    'Cumulative total': 'test_cumulTotal'
})
testing_df['date'] = pd.to_datetime(testing_df['date'])

testing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56264 entries, 0 to 56263
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   CountryCode            56264 non-null  object        
 1   date                   56264 non-null  datetime64[ns]
 2   test_cumulTotal        46719 non-null  float64       
 3   test_7dayMA            54542 non-null  float64       
 4   test_shortPosRate      51040 non-null  float64       
 5   test_shortTestperCase  50410 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 2.6+ MB


In [6]:
vaccination_df = controller.get_df_from_sql(
    "select iso_code, date, people_vaccinated, people_fully_vaccinated, daily_vaccinations "
    "from covid_info.OWID_vaccination_data"
).rename(columns={
    'iso_code': 'CountryCode',
    'people_vaccinated': 'vac_first',
    'people_fully_vaccinated': 'vac_second',
    'daily_vaccinations': 'vac_daily'
})
vaccination_df['date'] = pd.to_datetime(vaccination_df['date'])

In [7]:
mobility_df = controller.get_df_from_sql(
    "select CountryCode, date, "
    "retail_and_recreation_percent_change_from_baseline, "
    "grocery_and_pharmacy_percent_change_from_baseline, "
    "parks_percent_change_from_baseline, "
    "transit_stations_percent_change_from_baseline, "
    "workplaces_percent_change_from_baseline, "
    "residential_percent_change_from_baseline "
    "from covid_info.Google_Mobility "
    "where CountryCode != 'not found'"
).rename(columns={
    'retail_and_recreation_percent_change_from_baseline': 'mob_retailRecreation',
    'grocery_and_pharmacy_percent_change_from_baseline': 'mob_groceryPharmacy',
    'parks_percent_change_from_baseline': 'mob_parks',
    'transit_stations_percent_change_from_baseline': 'mob_transit',
    'workplaces_percent_change_from_baseline': 'mob_work',
    'residential_percent_change_from_baseline': 'mob_resident'
})
mobility_df['date'] = pd.to_datetime(mobility_df['date'])

In [8]:
countries_with_all_data = set(confirmed_df['CountryCode']) \
& set(testing_df['CountryCode']) \
& set(vaccination_df['CountryCode']) \
& set(mobility_df['CountryCode'])

for dels in ['CPV', 'CZE', 'FRA', 'LBY', 'SWE']:
    countries_with_all_data.remove(dels)

In [9]:
owid_df = testing_df.merge(vaccination_df, 
                           on=['CountryCode', 'date'],
                           how='outer')
owid_df = owid_df[owid_df['CountryCode'].isin(countries_with_all_data)]
# owid_df['date'] = pd.to_datetime(owid_df['date'])

In [10]:
mobility_df = mobility_df[mobility_df['CountryCode'].isin(countries_with_all_data)]

In [11]:
total_df = covid_df.merge(owid_df, on=['CountryCode', 'date'], how='outer')
total_df = total_df.merge(mobility_df, on=['CountryCode', 'date'], how='outer')

In [13]:
total_df.to_csv('./total_raw.csv')
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104791 entries, 0 to 104790
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   CountryCode            104791 non-null  object        
 1   date                   104791 non-null  datetime64[ns]
 2   covid_confirmed        104702 non-null  float64       
 3   covid_deaths           104702 non-null  float64       
 4   covid_recovered        104702 non-null  float64       
 5   test_cumulTotal        41225 non-null   float64       
 6   test_7dayMA            45215 non-null   float64       
 7   test_shortPosRate      42982 non-null   float64       
 8   test_shortTestperCase  42474 non-null   float64       
 9   vac_first              12066 non-null   float64       
 10  vac_second             10207 non-null   float64       
 11  vac_daily              17665 non-null   float64       
 12  mob_retailRecreation   54393 non-null   floa